In [ ]:
import urllib3
from bs4 import BeautifulSoup
import pymongo 
from pprint import pprint 
import pandas as pd
import requests
from splinter import Browser
import json
import tweepy
import pymongo
import os
import time
from pprint import pprint
from selenium import webdriver





In [ ]:
def TargetPlayerInfo(rank):
        rank = rank + 1
        
        xpath1 = f'/html/body/table[2]/tbody/tr[{rank}]/td[2]/a'
        xpath2 = f'/html/body/table[2]/tbody/tr[{rank}]/td[3]/span'
        xpath3 = f'/html/body/table[2]/tbody/tr[{rank}]/td[4]/img'
        xpath4 = f'/html/body/table[2]/tbody/tr[{rank}]/td[5]'
        
        browser = Browser('chrome', headless=False)
        url = 'https://www.goratings.org/en/'
        browser.visit(url)
        preserve_link = browser.find_by_xpath(xpath1) ## postion as ranking + 1
        
        player_gender = browser.find_by_xpath(xpath2)
        if player_gender.text == '♂':
            gender = "Male"
        else:
            gender = "Female"
        
        player_natinality = browser.find_by_xpath(xpath3)
        
        players_ranking = browser.find_by_xpath(xpath4)
        
        firstLayer = {
            'targeted link' : preserve_link['href'],
            'targeted name' : preserve_link.text,
            "targeted gender" : gender,
            "Targeted Natinality" :  player_natinality['alt'], 
            "Targeted Rank" : players_ranking.text
            
        }
        time.sleep(3)
        
        browser.quit()
        time.sleep(3)
        # start another link scraping
        driver = Browser('chrome', headless=False)
        url1 = firstLayer['targeted link'] 
        driver.visit(url1)
        html = driver.html
#         soup = BeautifulSoup(html, "html.parser")
        table = driver.find_by_xpath('/html/body/table[1]/tbody')
        info = []
        for i in table.find_by_tag("td"):
            info.append(i.text)


        
        firstLayer["Wins"] = info[0]
        firstLayer["Losses"] = info[1]
        firstLayer["Total"] = info[2]
        firstLayer["Birthday"] = info[-1]
        
        return firstLayer
    



In [ ]:
# game record and opponent info
def GamingRecordDetails(rank):
    
        rank = rank + 1
        
        xpath1 = f'/html/body/table[2]/tbody/tr[{rank}]/td[2]/a'
        browser = Browser('chrome', headless=False)
        url = 'https://www.goratings.org/en/'
        browser.visit(url)
        preserve_link = browser.find_by_xpath(xpath1) ## postion as ranking + 1
        
        SecondLayer = {
            'targeted link' : preserve_link['href'],
            'targeted gamer' : preserve_link.text,
        }
        time.sleep(3)
        
        browser.quit()
       
        time.sleep(3)
        # start another link scraping
        driver = Browser('chrome', headless=False)
        url1 = SecondLayer['targeted link'] 
        driver.visit(url1)
        html = driver.html
        table = driver.find_by_xpath('/html/body/table[2]/tbody')


        info = []

        #table.find_by_tag('td')
        for tr in table.find_by_tag('tr'):
            info.append(tr.text.split(" "))
            
        data = pd.DataFrame(info)

        data = data.drop([8, 9, 10], axis=1).drop([0])
        data['Opponant Name'] = data[[4, 5]].apply(lambda x: ' '.join(x), axis=1)
        data = data.drop([4,5], axis=1)
        data
        data = data.rename(columns = {0: "Date", 1: "Gamer Rating", 2: "Color", 3: "Result", 'Opponant Name': "Opponent Name",  6: "Opponent Ranking",
                             7: "Opponent Gender"})

        df = data[["Date", "Gamer Rating","Color", "Result", "Opponent Name",  "Opponent Ranking","Opponent Gender"]]

        df.head()
        final_dict = {

            'Date' : list(df['Date']),
            'Gamer Rating': list(df['Gamer Rating']),
            'Color': list(df['Color']),
            'Result': list(df['Result']),
            'Opponent Name': list(df['Opponent Name']),
            'Opponent Ranking': list(df['Opponent Ranking']),
            'Opponent Gender':  list(df['Opponent Gender'])

              }
        
        driver.quit()

        return final_dict


        